In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()


In [2]:
spark

In [7]:
sc=spark.sparkContext

# map
#Return a new distributed dataset formed by passing each element of the source through a function func.

In [8]:
rdd = sc.parallelize(["b", "a", "c"])
#sorted(rdd.map(lambda x: (x, 1)).collect())
rdd.map(lambda x: (x, 1)).collect()


[('b', 1), ('a', 1), ('c', 1)]

# filter
#Return a new dataset formed by selecting those elements of the source on which func returns true.

In [10]:
rdd = sc.parallelize([1, 2, 3, 4, 5,6])
rdd.filter(lambda x: x % 2 == 0).collect()


[2, 4, 6]

# flatMap
#Similar to map, but each input item can be mapped to 0 or more output items (so func should return a Seq rather than a single item).

In [ ]:
rdd = sc.parallelize([2, 3, 4])
sorted(rdd.flatMap(lambda x: range(1, x)).collect())

# mapPartitions
#Similar to map, but runs separately on each partition (block) of the RDD, so func must be of type Iterator<T> => Iterator<U> when running on an RDD of type T

In [11]:
rdd = sc.parallelize([1, 2, 3, 4], 2)
def f(iterator): yield sum(iterator)
rdd.mapPartitions(f).collect()

[3, 7]

# mapPartitionsWithIndex
#Similar to mapPartitions, but also provides func with an integer value representing the index of the partition, so func must be of type (Int, Iterator<T>) => Iterator<U> when running on an RDD of type T.

In [12]:
rdd = sc.parallelize([1, 2, 3, 4], 4)
def f(splitIndex, iterator): yield splitIndex
rdd.mapPartitionsWithIndex(f).sum()

6

# sample
#Sample a fraction fraction of the data, with or without replacement, using a given random number generator seed.

In [13]:
rdd = sc.parallelize(range(100), 4)
6 <= rdd.sample(False, 0.1, 81).count() <= 14

True

# union
#Return a new dataset that contains the union of the elements in the source dataset and the argument.

In [14]:
rdd = sc.parallelize([1, 1, 2, 3])
rdd.union(rdd).collect()


[1, 1, 2, 3, 1, 1, 2, 3]

# intersection(otherDataset)
#Return a new RDD that contains the intersection of elements in the source dataset and the argument.

In [15]:
rdd1 = sc.parallelize([1, 10, 2, 3, 4, 5])
rdd2 = sc.parallelize([1, 6, 2, 3, 7, 8])
rdd1.intersection(rdd2).collect()

[1, 2, 3]

# distinct([numPartitions]))
#Return a new dataset that contains the distinct elements of the source dataset.

In [16]:
sorted(sc.parallelize([1, 1, 2, 3]).distinct().collect())

[1, 2, 3]

# groupByKey([numPartitions])
#When called on a dataset of (K, V) pairs, returns a dataset of (K, Iterable<V>) pairs.
Note: If you are grouping in order to perform an aggregation (such as a sum or average) over each key, using reduceByKey or aggregateByKey will yield much better performance.
Note: By default, the level of parallelism in the output depends on the number of partitions of the parent RDD. You can pass an optional numPartitions argument to set a different number of tasks.

In [17]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
sorted(rdd.groupByKey().mapValues(len).collect())

[('a', 2), ('b', 1)]

# reduceByKey(func, [numPartitions])
#When called on a dataset of (K, V) pairs, returns a dataset of (K, V) pairs where the values for each key are aggregated using the given reduce function func, which must be of type (V,V) => V. Like in groupByKey, the number of reduce tasks is configurable through an optional second argument.

In [18]:
from operator import add
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
sorted(rdd.reduceByKey(add).collect())

[('a', 2), ('b', 1)]

# aggregate
# When called on a dataset of (K, V) pairs, returns a dataset of (K, U) pairs where the values for each key are aggregated using the given combine functions and a neutral "zero" value. Allows an aggregated value type that is different than the input value type, while avoiding unnecessary allocations. Like in groupByKey, the number of reduce tasks is configurable through an optional second argument.

In [1]:
seqOp = (lambda x, y: (x[0] + y, x[1] + 1))
combOp = (lambda x, y: (x[0] + y[0], x[1] + y[1]))
sc.parallelize([1, 2, 3, 4]).aggregate((0, 0), seqOp, combOp)
sc.parallelize([]).aggregate((0, 0), seqOp, combOp)

NameError: name 'sc' is not defined

# sortByKey([ascending], [numPartitions])
#When called on a dataset of (K, V) pairs where K implements Ordered, returns a dataset of (K, V) pairs sorted by keys in ascending or descending order, as specified in the boolean ascending argument.

In [ ]:
tmp = [('a', 1), ('b', 2), ('1', 3), ('d', 4), ('2', 5)]
sc.parallelize(tmp).sortByKey().first()
sc.parallelize(tmp).sortByKey(True, 1).collect()

# join(otherDataset, [numPartitions])
# When called on datasets of type (K, V) and (K, W), returns a dataset of (K, (V, W)) pairs with all pairs of elements for each key. Outer joins are supported through leftOuterJoin, rightOuterJoin, and fullOuterJoin.

In [ ]:
x = sc.parallelize([("a", 1), ("b", 4)])
y = sc.parallelize([("a", 2), ("a", 3)])
sorted(x.join(y).collect())

# cogroup(otherDataset, [numPartitions])
# When called on datasets of type (K, V) and (K, W), returns a dataset of (K, (Iterable<V>, Iterable<W>)) tuples. This operation is also called groupWith.

In [ ]:
x = sc.parallelize([("a", 1), ("b", 4)])
y = sc.parallelize([("a", 2)])
[(x, tuple(map(list, y))) for x, y in sorted(list(x.cogroup(y).collect()))]

# cartesian(otherDataset)
# When called on datasets of types T and U, returns a dataset of (T, U) pairs (all pairs of elements).

In [ ]:
rdd = sc.parallelize([1, 2])
sorted(rdd.cartesian(rdd).collect())

# pipe(command, [envVars])
# Pipe each partition of the RDD through a shell command, e.g. a Perl or bash script. RDD elements are written to the process's stdin and lines output to its stdout are returned as an RDD of strings.

In [ ]:
sc.parallelize(['1', '2', '', '3']).pipe('cat').collect()

# coalesce(numPartitions)
# Decrease the number of partitions in the RDD to numPartitions. Useful for running operations more efficiently after filtering down a large dataset.

In [ ]:
sc.parallelize([1, 2, 3, 4, 5], 3).glom().collect()
sc.parallelize([1, 2, 3, 4, 5], 3).coalesce(1).glom().collect()

# repartition(numPartitions)
# Reshuffle the data in the RDD randomly to create either more or fewer partitions and balance it across them. This always shuffles all data over the network.

In [ ]:
rdd = sc.parallelize([1,2,3,4,5,6,7], 4)
sorted(rdd.glom().collect())
len(rdd.repartition(2).glom().collect())
len(rdd.repartition(10).glom().collect())

# repartitionAndSortWithinPartitions(partitioner)
# Repartition the RDD according to the given partitioner and, within each resulting partition, sort records by their keys. This is more efficient than calling repartition and then sorting within each partition because it can push the sorting down into the shuffle machinery.

In [ ]:
rdd = sc.parallelize([(0, 5), (3, 8), (2, 6), (0, 8), (3, 8), (1, 3)])
rdd2 = rdd.repartitionAndSortWithinPartitions(2, lambda x: x % 2, True)
rdd2.glom().collect()
